#### CT-208 - Matemática Computacional


##### 1. Covariance Matrix Adaptation Evolution Strategy (CMA-ES)

- Modelo Evolutivo de 2ª Ordem (gradiente e curvatura);
- Modelos Evolutivos geram populações através de uma distribuição normal multivariada em $\mathbb{R}^n$, medem a performance de cada indivíduo, selecionam os p melhores para atualizar a média da distribuição para a próxima iteração.
- CMA atualiza não só a **média**, como também a **matriz de covariância** que gera a população.
- O Step-size também é atualizado a cada iteração de acordo com os parâmetros $p_\sigma$ (Exploration) e $p_c$ (Exploitation).
- $p_\sigma$ e $p_c$ iniciam com pesos iguais e vão atualizando também de acordo com a performance da população.

##### Parâmetros:

- $\lambda$: Tamanho da população
- $m$: Média da distribuição
- $C$: Mátriz covariância 3x3
- $p_\sigma$: Vetor Search path (Exploration) 
- $p_c$: Vetor Evolution path (Exploitation)
- $\sigma$: Step-size

##### Exemplo:
<img src="CMA.png" alt="cmaes" class="bg-primary mb-1" width="600px" style={align:center}>

##### Output de Sample:

sample = $[position_x, position_y, [-1,1]]$

##### Função de Custo

$Cost = intersection*intersection\_cost + distance\_to\_origin*distance\_cost$

##### Updates

- Resolvido bugs de tamanho de peças
- Adição de rotação de $^+_-90$ graus
- Execução para todas as peças

##### Próximos passos

- Otimizar parâmetros
- Resolver bug de exportar produto final para SVG que está rotacionando os componentes

##### 2. Parsing SVG Files

In [1]:
import os
import re
import numpy as np

from PIL             import Image
from parseSvgFiles   import parseSVG
from convertSvg2Png  import convertSVG2PNG
from convertPng2Svg  import convertPNG2SVG
from placeComponent  import ComponentPlacing
from utils           import binarray2jpg, png2bin, selectComponentToPlace, save_object

In [2]:
# Sheet size
PAGE_SIZE = (210,297) 

# Define which project to optimize
#svg_name = 'example'
#svg_name = 'harry-potter'
svg_name = 'e190-e2'

Parsing each SVG file into separate components

In [3]:
# Select figure in folder and parse SVG file into separate components 
parseSVG(svg_name)

Components from e190-e2 separated with success!


Convert each SVG component into a PNG equivalent

In [4]:
# Parse each SVG component into PNG equivalent 
convertSVG2PNG(svg_name)

Components from e190-e2 transformed with success!


In [5]:
# Components and file paths
components_path = './' + svg_name + '/components_png'
test_results_path = './' + svg_name + '/test_results'
model_path = './' + svg_name + '/results.pkl'

print("\nPaths:")
print(components_path)
print(test_results_path)


Paths:
./e190-e2/components_png
./e190-e2/test_results


In [6]:
# Finds the component png file in the directory
files = os.listdir(components_path)
pngs = list(filter(lambda name: re.match('.*\.png', name), files))

Optimizer Parameters

In [7]:
# Optimizer parameters
iterations = 300
sigma = 1.0
population = 12

In [8]:
# Creates a list of pages and append a page of (height, lenght) px
pages = []
pages.append(np.zeros(PAGE_SIZE, dtype=int))
# Transforms each png component into a binary matrix
# List of binary matrix of components and its name
components = [[png2bin(components_path + '/' + png), png] for png in pngs] 
# Creates a list of objects of class ComponentPlacing
CP_list = [ComponentPlacing(pages[0], component, sigma=sigma, population=population, num_iterations=iterations) for component in components]


(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=750530, Wed Jul 13 00:00:40 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=793572, Wed Jul 13 00:00:40 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=808212, Wed Jul 13 00:00:40 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=749189, Wed Jul 13 00:00:40 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=823578, Wed Jul 13 00:00:40 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=750432, Wed Jul 13 00:00:40 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=797673, Wed Jul 13 00:00:40 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=736212, Wed Jul 13 00:00:40 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=779160, Wed Jul 13 00:00:40 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=832469, Wed Jul 13 00:00:40 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=813411, Wed Jul 13 00:00:40 2022)

In [9]:
# Place components on page
results = []
page_idx = 0
max_attempts = 100
num_components = len(CP_list)


In [10]:
while CP_list:
    attempt = 0     # counter of times that try to fit a component in a page
    # This loop tries to fit a component on current page    
    while attempt < max_attempts:
        # Selects one aleatory component
        idx = selectComponentToPlace(CP_list)
        CP = CP_list[idx]
        # Finds the best position
        CP.FindCandidatePosition()
        # Checks if there is no intersection and place the compoponent if not
        if CP.NoIntersection():
            pages[page_idx] = CP.PlaceComponentInPage()
            # removes the placed component from the list and store in results list
            results.append(CP_list.pop(idx)) 
            num_components -= 1
            print(('Current page: ' + str(page_idx+1)).ljust(18) + 'Components remaining: ' + str(num_components)+'...')
            
            if CP_list: 
                attempt = 0
            else: 
                break
        else: 
            attempt += 1
            CP.resetParameters() # sorts a new initial point
            
    # Prepares a new page if necessary
    if CP_list:
        pages.append(np.zeros(PAGE_SIZE, dtype=int))
        page_idx += 1
        for CP in CP_list:
            # sorts new initial point for the objects
            CP.page = pages[page_idx]
            CP.page_idx = page_idx
            CP.resetParameters()


Current page: 1   Components remaining: 54...
Current page: 1   Components remaining: 53...
Current page: 1   Components remaining: 52...
Current page: 1   Components remaining: 51...
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=835069, Wed Jul 13 00:00:44 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=795184, Wed Jul 13 00:00:45 2022)
Current page: 1   Components remaining: 50...
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=773004, Wed Jul 13 00:00:47 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=847113, Wed Jul 13 00:00:47 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=811701, Wed Jul 13 00:00:48 2022)
Current page: 1   Components remaining: 49...
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=816520, Wed Jul 13 00:00:50 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=704507, Wed Jul 13 00:00:51 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=841739, Wed Jul 13 00:00:51 20

In [11]:
# Remove all png files from the folder before storing the results
files = os.listdir(test_results_path)
for file in files:
    os.remove(test_results_path + '/' + file)


In [12]:

# Saves pages
for idx,page in enumerate(pages):
    image = binarray2jpg(page).astype(np.uint8)
    image = Image.fromarray(image)
    image.save(test_results_path + '/page' + str(idx).zfill(2) + '.png')

save_object(results, model_path)

print('Finished optimization.')

Finished optimization.


In [13]:
# Parse each PNG file into SVG file 
convertPNG2SVG(svg_name)

Success! See results in folder /e190-e2/results/
